In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Dealing with the data

In [6]:
data = pd.read_csv('bodyfat.csv')
data.shape
data.head()

,Density,BodyFat,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.0708,12.3,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,1.0853,6.1,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,1.0414,25.3,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,1.0751,10.4,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,1.0340,28.7,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


In [8]:
#Standardize header names.
def coloumn_names_lwr(data):
    data.columns = map(str.lower, data.columns)
    data.head()
    #data.columns = [x.lower() for x in data.columns]
    
def coloumn_names_captl(data):
    data.columns = map(str.capitalize, data.columns)
    data.head()
    #data.columns = [x.capitalize() for x in data.columns]    
'''
def replace_whitespaces(data):
    cols = []
    for col in data.columns:
        cols.append(col.replace(' ', '_'))
    data.columns = cols
    return data
'''    


In [ ]:
def check_missing_data(data):
    print(f"zero values in the dataset\n: {data.isnull().sum() * 100 / len(data)}")
    print(f"unknown values in the dataset:{data.isna().sum() * 100 / len(data)}")
    print("\n")
    print("do you wish to clean your data set ?")
    userinput = input("press 1 to clean, press 0 to skip")
    if userinput == 1:
        print("cleaning your data set ... ")
    elif userinput != 0:
        print("Cleaning skipped..")    


In [ ]:
#numercial and categorical data

numerical_values = pd.DataFrame()
continuous_values = pd.DataFrame()
categorical_values = pd.DataFrame()

def check_dataypes(data):
    numerical = data.select_dtypes(np.number)
    numerical_continuous = data.select_dtypes(include=['float64'])
    categorical = data.select_dtypes(object)
    return numerical,numerical_continuous,categorical


Omitting the density coloumn on purpose because to predict the body fat percentage of a person based on the circumferential measurements already available, which is good enough to predict with so much available data.

I will, create Model only with these data, and predict again with my personal data set for the information randomly to validate the model.

Fat (%) = [(4*95/density) -4.51 x 100]

source:https://www.cambridge.org/core/services/aop-cambridge-core/content/view/DA80501B784742B9B2F4F454BDEE923B/S0007114567000728a.pdf/the-assessment-of-the-amount-of-fat-in-the-human-body-from-measurements-of-skinfold-thickness.pdf


# Exploring the data

In [ ]:
#converting the dataset to metric system
def convert_weight_kg(data):
    data['weight']=data['weight'].apply(lambda x : round((x * 0.453),2))
    
def convert_inch_to_cm(data):
    #drop weight, density and body fat percentage
    # 12 inches --> 30 cm
    #formula X cm = [30/12] * input inches
    df_drops = data[['density','bodyfat','age','weight']]
    df2 = data.drop(['density','bodyfat','age','weight'], axis=1)
    df2 = df2.apply(lambda x : x * 2.5)
    data = pd.concat([df_drops, df2], axis=1, join="outer")
    return data
    

Finding the Relationships:

I am concerned about the correlation between the Label = 'Bodyfat%' and the features = [weight,height,Chest,abdomen, hip, Bicep,Thigh]

so I am going to find the Correation between them, by dropping the rest as follows

In [ ]:
def find_correlation(data):
    df_corr = data.drop(['density','age'], axis=1)
    sns.heatmap(df_corr.corr())
    #return df_corr
    

In [ ]:
#pd.set_option('display.max_columns', None)

In [ ]:
# Fucntion calls and declarations
numerical_values = pd.DataFrame()
continuous_values = pd.DataFrame()
categorical_values = pd.DataFrame()
data_corr = pd.DataFrame()
coloumn_names_lwr(data)
check_missing_data(data)
numerical_values,continuous_values,categorical_values = check_dataypes(data)
convert_weight_kg(data)
convert_inch_to_cm(data)
find_correlation(numerical_values)#input numerical values
data

#numerical_values.head(20)


# Processing Data

X-y splitting
Normalizing (numericals)
Concating DataFrames ...

# Modelling (decision pending..)